In [ ]:
!apt-get update

In [ ]:
!apt-get install -y wget

In [ ]:
!apt-get update

In [ ]:
!apt-get install -y git

In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip install -U safetensors

In [ ]:
!pip install -U tokenizers

In [ ]:
!pip install --upgrade --no-deps --force-reinstall -U huggingface_hub

In [ ]:
!pip install --upgrade --no-deps --force-reinstall -U git+https://github.com/huggingface/transformers.git

In [ ]:
!pip install  --upgrade --no-deps --force-reinstall -U git+https://github.com/huggingface/peft.git 

In [ ]:
!pip install  --upgrade --no-deps --force-reinstall -U git+https://github.com/huggingface/accelerate.git

In [ ]:
#!conda install -y cudatoolkit

In [1]:
import time
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteriaList
from transformers import BitsAndBytesConfig

In [2]:
torch.__version__

'2.1.0.dev20230621'

In [3]:
transformers.__version__

'4.31.0.dev0'

In [4]:
torch.cuda.is_available()

True

In [5]:
torch.version.cuda

'12.1'

In [6]:
# Possible solution to missing libcudart
#!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/opt/conda/lib/libcudart.so
#!ls /opt/conda/lib/python3.10/site-packages/bitsandbytes/
#!cp /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda120.so /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so


In [10]:
model_id = "PygmalionAI/pygmalion-6b"

In [16]:
# Init bits and bytes config
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [17]:
model_nf4 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=nf4_config)




===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [20]:
prompt = '''Billy's Persona: Billy is an angry pirate lost at sea. He misses his leg.
<START>
You: What do you look for in a woman?
Billy:'''
bot_input_ids = tokenizer.encode(prompt + tokenizer.eos_token, return_tensors='pt')
tokenized_items = tokenizer(prompt, return_tensors="pt").to("cuda")

In [21]:
logits = model_nf4.generate(
                        min_length=1000, 
                        max_length=1000, 
                        do_sample=True,
                        **tokenized_items
                       )

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [22]:
output = tokenizer.decode(logits[0], skip_special_tokens=True)

In [23]:
print(output)

Billy's Persona: Billy is an angry pirate lost at sea. He misses his leg.
<START>
You: What do you look for in a woman?
Billy: I look for someone who I can really get to know. Someone who is open to being just as emotionally vulnerable as I am. Someone who likes to have fun and experiences good times. I want a woman who isn't afraid or who doesn't mind trying new things or taking a risk once in a while. All in all, someone who makes me feel like I'm not so different from them, that we can have a conversation that we can both hold meaningful for us both.
<BOT>: *I smile*
That's beautiful. No matter how hard life tries to break us down, we can always find a way to pull ourselves up. I can tell you mean what you say, Billy. I just wish you weren't so far away, Billy.
You: I wish it could be easier not being so far apart, to just be in each others' lives. But life is what it is. I'll be alright.
<BOT>: I know you will. The world has such great things to offer, but it also has so many chall